In [28]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json
load_dotenv()

True

In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name='fiancerag', 
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [27]:
data = json.load(open('/Users/ceo/Desktop/headstarter_SWE/Final_Project/Buddyfin-AI/src/data/reviews.json'))

[{'company': 'GameStop Corp.',
  'ticker': 'GME',
  'reason': 'Declining sales and struggling to adapt to digital game sales'},
 {'company': 'Bed Bath & Beyond Inc.',
  'ticker': 'BBBY',
  'reason': 'Fierce competition from online retailers and declining sales'},
 {'company': 'J.C. Penney Company Inc.',
  'ticker': 'JCPNQ',
  'reason': 'Bankruptcy and struggling to compete with online retailers'},
 {'company': 'Sears Holdings Corp.',
  'ticker': 'SHLDQ',
  'reason': 'Bankruptcy and struggling to compete with online retailers'},
 {'company': 'Coty Inc.',
  'ticker': 'COTY',
  'reason': 'Declining sales and struggling to compete with online beauty retailers'},
 {'company': "Macy's Inc.",
  'ticker': 'M',
  'reason': 'Declining sales and struggling to compete with online retailers'},
 {'company': 'Ford Motor Co.',
  'ticker': 'F',
  'reason': 'Declining sales and struggling to adapt to electric vehicle market'},
 {'company': 'General Motors Co.',
  'ticker': 'GM',
  'reason': 'Declining s

In [54]:
processed_data = []
client = OpenAI()
#We're going to embed the best_stocks and poor_stocks advice
for review in data['best_stocks']:
    response = client.embeddings.create(
        input= review['company'],
        model='text-embedding-3-small'
    )
    embedding = response.data[0].embedding

    processed_data.append({
        'values': embedding,
        'id': review['company'],
        'metadata': {
            'type': 'best_stock',
            'company': review['company'],
            'ticker': review['ticker'],
            'analysis': review['analyst_recommendation'],
            'reason': review['reason']
        },
    })

for data_value in data['poor_stocks']:
    response = client.embeddings.create(
        input= data_value['company'],
        model='text-embedding-3-small'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        'values': embedding,
        'id': data_value['company'],
        'metadata': {
            'type': 'poor_stock',
            'company': data_value['company'],
            'ticker': data_value['ticker'],
            'analysis': data_value.get('analyst_recommendation', None),
            'reason': data_value['reason'],
        },
    })


   
    

KeyError: 'analyst_recommendation'

In [38]:
processed_data[0]


{'values': [-0.041350596,
  -0.059299327,
  0.059573356,
  0.07393234,
  -0.027717778,
  0.004004212,
  -0.02473089,
  0.06790376,
  0.004740658,
  -0.020949325,
  0.012283237,
  -0.021593286,
  -0.016852628,
  -0.0052715843,
  0.014016454,
  0.0057579814,
  -0.025333747,
  -0.012166776,
  -0.031293824,
  -0.044967744,
  0.015578405,
  0.034088895,
  0.029978497,
  -0.007193195,
  -0.034390323,
  0.017770616,
  -0.043816835,
  -0.015551003,
  -0.0022042005,
  -0.0062991837,
  0.05006464,
  -0.029978497,
  0.0403367,
  0.034609545,
  -0.0069191684,
  0.0372676,
  0.011611872,
  0.07743988,
  0.011522814,
  -0.02989629,
  0.024086928,
  -0.00403504,
  0.023621082,
  -0.0020106693,
  0.0049804314,
  -0.005432575,
  -0.03126642,
  -0.039048772,
  0.0020466351,
  -0.004182329,
  -0.038774747,
  -0.0059395237,
  -0.018551592,
  0.028909793,
  -0.033760063,
  0.030827979,
  0.04099436,
  0.04200826,
  0.016838927,
  0.019181853,
  -0.039816048,
  -0.029868886,
  0.03280097,
  0.0015979169,
  

In [48]:
index = pc.Index('fiancerag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)


{'upserted_count': 13}